## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,55.06,85,4,1.01,clear sky
1,1,Deputatskiy,RU,69.3000,139.9000,-21.35,73,68,3.42,broken clouds
2,2,Avarua,CK,-21.2078,-159.7750,78.85,89,75,13.80,broken clouds
3,3,Dikson,RU,73.5069,80.5464,4.48,93,100,17.00,light snow
4,4,Fortuna,US,40.5982,-124.1573,49.93,97,20,5.01,few clouds


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input('Minimum temperature you would like for your trip? '))
max_temp=float(input('Maximum temperature you would like for your trip? '))

Minimum temperature you would like for your trip? 70
Maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df=city_data_df.loc[(city_data_df['Max Temp']>=min_temp)&(city_data_df['Max Temp']<=max_temp)]
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Avarua,CK,-21.2078,-159.7750,78.85,89,75,13.80,broken clouds
5,5,Henties Bay,NaN,-22.1160,14.2845,71.85,84,63,5.61,broken clouds
13,13,Rikitea,PF,-23.1203,-134.9692,78.40,75,100,9.33,overcast clouds
14,14,Rio Cuarto,AR,-33.1307,-64.3499,82.17,61,40,16.11,scattered clouds
15,15,Axim,GH,4.8699,-2.2405,81.91,61,70,6.91,broken clouds
...,...,...,...,...,...,...,...,...,...,...
688,688,Oum Hadjer,TD,13.2954,19.6966,71.92,21,31,10.45,scattered clouds
690,690,Luanda,AO,-8.8368,13.2343,80.60,65,20,9.22,few clouds
691,691,Dhidhdhoo,MV,6.8833,73.1000,81.27,67,37,9.42,scattered clouds
693,693,Manaus,BR,-3.1019,-60.0250,86.50,74,40,6.91,scattered clouds


In [14]:
# 4a. Determine if there are any empty rows.
clean_df[clean_df.isna().any(axis=1)]

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Henties Bay,NaN,-22.1160,14.2845,71.85,84,63,5.61,broken clouds
284,284,Karasburg,NaN,-28.0167,18.7500,73.94,54,100,19.04,light rain
466,466,Luderitz,NaN,-26.6481,15.1594,73.87,68,60,14.18,broken clouds
666,666,Opuwo,NaN,-18.0607,13.8400,74.23,32,100,7.94,overcast clouds


In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=clean_df.dropna()

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,
13,Rikitea,PF,78.40,overcast clouds,-23.1203,-134.9692,
14,Rio Cuarto,AR,82.17,scattered clouds,-33.1307,-64.3499,
15,Axim,GH,81.91,broken clouds,4.8699,-2.2405,
16,Busselton,AU,79.09,broken clouds,-33.6500,115.3333,
17,Butaritari,KI,81.70,broken clouds,3.0707,172.7902,
18,Kathu,TH,78.62,clear sky,7.9178,98.3332,
20,Mahebourg,MU,80.89,scattered clouds,-20.4081,57.7000,
22,Beloha,MG,71.46,overcast clouds,-25.1667,45.0500,
24,Port Alfred,ZA,76.14,clear sky,-33.5906,26.8910,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
no_name_hotel= []
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat= row['Lat']
    lng=row['Lng']
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6c. Get latitude and longitude from DataFrame
    hotel_df[["Lat", "Lng"]]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        pass
        print('IndexError',index)
        #indexes rows to be dropped with no Hotel Name
        if index not in no_name_hotel:
            no_name_hotel.append(index)

print(no_name_hotel)
hotel_df

IndexError 22
IndexError 44
IndexError 175
IndexError 204
IndexError 247
IndexError 289
IndexError 299
IndexError 302
IndexError 313
IndexError 319
IndexError 381
IndexError 422
IndexError 446
IndexError 460
IndexError 487
IndexError 517
IndexError 527
IndexError 597
IndexError 632
IndexError 658
IndexError 663
IndexError 682
IndexError 688
IndexError 696
[22, 44, 175, 204, 247, 289, 299, 302, 313, 319, 381, 422, 446, 460, 487, 517, 527, 597, 632, 658, 663, 682, 688, 696]


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,Paradise Inn
13,Rikitea,PF,78.40,overcast clouds,-23.1203,-134.9692,People ThankYou
14,Rio Cuarto,AR,82.17,scattered clouds,-33.1307,-64.3499,Grand Hotel Río Cuarto
15,Axim,GH,81.91,broken clouds,4.8699,-2.2405,Axim Beach Resort And Spa
16,Busselton,AU,79.09,broken clouds,-33.6500,115.3333,Observatory Guest House
...,...,...,...,...,...,...,...
688,Oum Hadjer,TD,71.92,scattered clouds,13.2954,19.6966,
690,Luanda,AO,80.60,few clouds,-8.8368,13.2343,Hotel Fórum
691,Dhidhdhoo,MV,81.27,scattered clouds,6.8833,73.1000,Dhidhoo beach
693,Manaus,BR,86.50,scattered clouds,-3.1019,-60.0250,Manaus Hotéis Millennium


In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.drop(no_name_hotel, inplace=False)
clean_hotel_df=hotel_df
clean_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,Paradise Inn
13,Rikitea,PF,78.40,overcast clouds,-23.1203,-134.9692,People ThankYou
14,Rio Cuarto,AR,82.17,scattered clouds,-33.1307,-64.3499,Grand Hotel Río Cuarto
15,Axim,GH,81.91,broken clouds,4.8699,-2.2405,Axim Beach Resort And Spa
16,Busselton,AU,79.09,broken clouds,-33.6500,115.3333,Observatory Guest House


In [22]:
# 8a. Create the output File (CSV)
output_file=('WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and Max. Temp. of: {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(50.0, 31.0), zoom_level=1.8)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))